In [ ]:
import os, csv, psycopg
from psycopg.rows import dict_row
from dotenv import load_dotenv

load_dotenv()

conn = psycopg.connect(
    host=os.getenv("PGHOST"),
    port=os.getenv("PGPORT"),
    dbname=os.getenv("PGDATABASE"),
    user=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    sslmode="require" if os.getenv("PGSSL", "true").lower() == "true" else "disable",
    row_factory=dict_row
)

In [ ]:
def load_table(csv_path, table, cols):
    with conn.cursor() as cur, open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            vals = [row[c] if row[c] != "" else None for c in cols]
            ph = ",".join(["%s"] * len(cols))
            cur.execute(
                f"INSERT INTO {table} ({','.join(cols)}) VALUES ({ph})", vals
            )
    conn.commit()

load_table("../data/customers.csv","customers",["name","state","age"])
load_table("../data/policies.csv","policies",["customer_id","policy_type","start_date","end_date","premium"])
load_table("../data/claims.csv","claims",["policy_id","date_of_loss","description","amount_claimed","status"])

In [1]:
import os
from openai import AzureOpenAI
from tools.sql_tool import run_parameterized

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION","2024-08-01-preview"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)
EMBED_MODEL = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

def retrieve_similar_notes(query_text: str, k: int = 5):
    resp = client.embeddings.create(model=EMBED_MODEL, input=query_text)
    emb = resp.data[0].embedding
    sql = (
        "SELECT note_id, claim_id, note_text, embedding <=> %s AS distance "
        "FROM claim_notes "
        "ORDER BY distance ASC "
        "LIMIT %s;"
    )
    return run_parameterized(sql, (emb, k))


ModuleNotFoundError: No module named 'tools'

In [ ]:
import os
print(os.path.exists("../data/customers.csv"))
